In [ ]:
!pip install --upgrade tensorflow tensorboard_plugin_profile

In [1]:
%load_ext tensorboard

In [1]:
# Load the runner
from hydra import initialize, compose

initialize(config_path="../pax/pax/conf", job_name="create_runner")
args = compose(config_name="config", overrides=["+experiment/rice=debug"])

/var/folders/0g/tbd8_nhx01s7spsllv70v4100000gn/T/ipykernel_91158/755042361.py:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="../pax/pax/conf", job_name="create_runner")


In [2]:
from pax.utils import Section
from pax.experiment import global_setup, env_setup, agent_setup, watcher_setup, runner_setup
import logging
from jax.lib import xla_bridge
from jax.config import config
config.update('jax_disable_jit', True)
print(f"Jax backend: {xla_bridge.get_backend().platform}")

"""Set up main."""
logger = logging.getLogger()
with Section("Global setup", logger=logger):
    save_dir = global_setup(args)

with Section("Env setup", logger=logger):
    env, env_params = env_setup(args, logger)

with Section("Agent setup", logger=logger):
    agent_pair = agent_setup(args, env, env_params, logger)

with Section("Watcher setup", logger=logger):
    watchers = watcher_setup(args, logger)

runner = runner_setup(args, env, agent_pair, save_dir, logger)


/Users/christophproschel/codingprojects/ism/thesis/main/pax/pax/experiment.py:705: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="conf", config_name="config")


Jax backend: cpu
Making network for Rice-v1
Making network for Rice-v1
Making network for Rice-v1
Making network for Rice-v1
Making network for Rice-v1
Making network for Rice-v1
ParameterReshaper: 5952 parameters detected for optimization.
Time to Compile Jax Methods: 2.4162590503692627 Seconds


In [ ]:
import jax
from jax.config import config
config.update('jax_disable_jit', True)

def run(num_iters):
    runner.run_loop(env_params, agent_pair, num_iters, watchers)

# COMPILE (3 times to make sure everything is compiled, you might need more or less 
# depending on your code) 
# for _ in range(3):
#     jax.block_until_ready(run(num_iters=1))

with jax.profiler.trace("/tmp/jax-trace", create_perfetto_link=True):
    jax.block_until_ready(runner.run_loop(env_params, agent_pair, 1, False))

Training
------------------------------
Number of Generations: 1
Number of Meta Episodes: 2
Population Size: 1000
Number of Environments: 1
Number of Opponent: 1
Log Interval: 5
------------------------------


In [ ]:
print("Done")

In [ ]:
%tensorboard --logdir=/tmp/jax-trace --port 6006